# NeOF Evaluation

In [1]:
import pathlib

import torch
import numpy as np
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
from PIL import Image

from neural_orientation_field.neof.dataset import NeOFImageDataset
from neural_orientation_field.neof.model import NeOFCoarseModel, NeOFFineModel
from neural_orientation_field.nerf.utils import cam_ray_from_pose
from neural_orientation_field.neof.utils import nerf_image_render, hair_dir_vec2color

In [2]:
# Use MPS device.
USE_DEVICE = "mps"

if USE_DEVICE == "mps" and torch.mps.is_available():
    device = torch.device("mps")
elif USE_DEVICE == "cuda" and torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

device

device(type='mps')

# Load Evaluation Dataset

In [3]:
# Input
BODY_MASK_PATH = "../../data/images/blender-hair-long-test/body_mask/"
HAIR_MASK_PATH = "../../data/images/blender-hair-long-test/hair_mask/"
HAIR_DIR_PATH = "../../data/images/blender-hair-long-test/hair_dir/"
CAMERA_PATH = "../../data/cameras/blender-hair-long-test/"
CHECKPOINT_PATH = "../../data/models/neof/blender-hair-long/"
OUTPUT_PATH = "../../data/evals/neof/blender-hair-long"

body_mask_path = pathlib.Path(BODY_MASK_PATH).resolve()
if not body_mask_path.exists():
    raise FileNotFoundError("Body mask path doesn't exist.")
hair_mask_path = pathlib.Path(HAIR_MASK_PATH).resolve()
if not hair_mask_path.exists():
    raise FileNotFoundError("Hair mask path doesn't exist.")
hair_dir_path = pathlib.Path(HAIR_DIR_PATH).resolve()
if not hair_dir_path.exists():
    raise FileNotFoundError("Hair directory path doesn't exist.")
camera_path = pathlib.Path(CAMERA_PATH).resolve()
if not camera_path.exists():
    raise FileNotFoundError("Camera path doesn't exist.")
checkpoint_path = pathlib.Path(CHECKPOINT_PATH).resolve()
if not checkpoint_path.exists():
    raise FileNotFoundError("Checkpoint path doesn't exist.")
output_path = pathlib.Path(OUTPUT_PATH).resolve()
if not output_path.exists():
    output_path.mkdir(parents=True, exist_ok=True)

frame_name_path = camera_path / "frame-names.txt"
cam_transform_path = camera_path / "camera-transforms.npy"
cam_param_path = camera_path / "camera-params.npy"

body_mask_path, hair_mask_path, hair_dir_path, frame_name_path, cam_transform_path, cam_param_path, checkpoint_path, output_path

(PosixPath('/Users/fangjun/Documents/stanford/cs229/final-project/data/images/blender-hair-long-test/body_mask'),
 PosixPath('/Users/fangjun/Documents/stanford/cs229/final-project/data/images/blender-hair-long-test/hair_mask'),
 PosixPath('/Users/fangjun/Documents/stanford/cs229/final-project/data/images/blender-hair-long-test/hair_dir'),
 PosixPath('/Users/fangjun/Documents/stanford/cs229/final-project/data/cameras/blender-hair-long-test/frame-names.txt'),
 PosixPath('/Users/fangjun/Documents/stanford/cs229/final-project/data/cameras/blender-hair-long-test/camera-transforms.npy'),
 PosixPath('/Users/fangjun/Documents/stanford/cs229/final-project/data/cameras/blender-hair-long-test/camera-params.npy'),
 PosixPath('/Users/fangjun/Documents/stanford/cs229/final-project/data/models/neof/blender-hair-long'),
 PosixPath('/Users/fangjun/Documents/stanford/cs229/final-project/data/evals/neof/blender-hair-long'))

In [4]:
with open(frame_name_path, "r") as frame_path_file:
    frame_names = frame_path_file.read().split("\n")
    body_mask_paths = [body_mask_path / frame_name for frame_name in frame_names]
    hair_mask_paths = [hair_mask_path / frame_name for frame_name in frame_names]
    hair_dir_paths = [hair_dir_path / frame_name for frame_name in frame_names]
with open(cam_transform_path, "rb") as cam_transform_file:
    cam_transforms = np.load(cam_transform_file)
with open(cam_param_path, "rb") as cam_param_file:
    cam_params = np.load(cam_param_file)

image_dataset = NeOFImageDataset(body_mask_paths, hair_mask_paths, hair_dir_paths, cam_params, cam_transforms)

## Load Model

In [5]:
model_params_file_name = "model_params.pth"
coarse_model_file_name = "coarse_final.pth"
fine_model_file_name = "fine_final.pth"

# Load model params.
model_params = torch.load(checkpoint_path / model_params_file_name, weights_only=True)

coarse_pos_encode = model_params["coarse_pos_encode"]
fine_pos_encode = model_params["fine_pos_encode"]
nc = model_params["nc"]
fc = model_params["fc"]
samples_per_ray = model_params["samples_per_ray"]
max_subd_samples = model_params["max_subd_samples"]

# Load model.
coarse_model = NeOFCoarseModel(num_encoding_functions=coarse_pos_encode)
coarse_model.to(device)
fine_model = NeOFFineModel(num_encoding_functions=fine_pos_encode)
fine_model.to(device)

coarse_model.load_state_dict(torch.load(checkpoint_path / coarse_model_file_name, weights_only=True, map_location=device))
fine_model.load_state_dict(torch.load(checkpoint_path / fine_model_file_name, weights_only=True, map_location=device))

<All keys matched successfully>

## Evaluate Model

In [6]:
# Runtime parameters.
ray_batch_size = 4096

In [7]:
test_vecs = []
pred_vecs = []
for (_, _, test_vec), cam_transform, (h, w), (f, cx, cy) in tqdm(image_dataset):
    cam_orig, cam_ray_world = cam_ray_from_pose(cam_transform, h, w, f, cx, cy)
    _, fine_pred = nerf_image_render(
        coarse_model,
        fine_model,
        cam_transform,
        cam_orig,
        cam_ray_world,
        ray_batch_size,
        nc,
        fc,
        samples_per_ray,
        max_subd_samples,
        coarse_pos_encode,
        fine_pos_encode,
        device
    )
    test_vecs.append(test_vec)
    pred_vecs.append(fine_pred)

  0%|          | 0/16 [00:00<?, ?it/s]

In [8]:
test_images = [hair_dir_vec2color(test_vec) for test_vec in test_vecs]
pred_images = [hair_dir_vec2color(pred_vec) for pred_vec in pred_vecs]

In [9]:
for i in tqdm(range(len(test_images))):
    test_image = Image.fromarray((test_images[i] * 255).astype(np.uint8))
    test_image.save(output_path / f"test_{i}.png")
    pred_image = Image.fromarray((pred_images[i] * 255).astype(np.uint8))
    pred_image.save(output_path / f"pred_{i}.png")

  0%|          | 0/16 [00:00<?, ?it/s]